# 3D-CNN

In [11]:
# 超参数设置
batch_size = 10
keep_rate = 0.8
train_epochs = 2   


# 普通参数设置
IMG_SIZE_PX = 50
SLICE_COUNT = 20
n_classes = 2
save_step=1


# 定义待输入数据
x = tf.placeholder('float')
y = tf.placeholder('float')


# 定义变量
#  5 x 5 x 5 patches, 1 channel, 32 features to compute.
weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
           # 5 x 5 x 5 patches, 32 channels, 64 features to compute.
           'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
           # 64 features
           'W_fc':tf.Variable(tf.random_normal([54080,1024])),
           'out':tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
          'b_conv2':tf.Variable(tf.random_normal([64])),
          'b_fc':tf.Variable(tf.random_normal([1024])),
          'out':tf.Variable(tf.random_normal([n_classes]))}

#image_shaped_input = tf.reshape(x,[-1,50 ,50 ,1])
#tf.summary.image('input',image_shaped_input, 10)



# 前向计算
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #  size of window  movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    
    #  image X  image Y  image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    forward = tf.matmul(fc, weights['out'])+biases['out']
    
    tf.summary.histogram('forward',forward)

    return forward




# 定义 prediction & loss_function & optimizer
prediction = convolutional_neural_network(x)

loss_function = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
tf.summary.scalar('loss',loss_function)
    
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss_function)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [12]:
def train_neural_network(x):
    
    much_data = np.load('muchdata-50-50-20.npy',allow_pickle= True)
    train_data = much_data[:-1]
    validation_data = much_data[-1:]

    # 创建保存模型目录   
    ckpt_dir="./ckpt_dir/"
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
        
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        merged_summary_op = tf.summary.merge_all()
        writer = tf.summary.FileWriter('C:/Users/cheng/Desktop/TFOD/Data_Science_Bowl_2017/log',sess.graph) #创建写入符
        
        successful_runs = 0
        total_runs = 0
        
        saver = tf.train.Saver() # 模型保存
            
        for epoch in range(train_epochs):
            epoch_loss = 0
            for data in train_data:
                total_runs += 1
                try:
                    # 读取批次数据
                    X = data[0]
                    Y = data[1]
                    # 执行批次训练
                    _, c = sess.run([optimizer, loss_function], feed_dict={x: X, y: Y})
                    
                    # 生成summary
                    summary_str = sess.run(merged_summary_op,feed_dict={x: X, y: Y})
                    writer.add_summary(summary_str, epoch)
                    
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    pass
            
            print('Epoch', epoch+1, 'completed out of',train_epochs,'loss:',epoch_loss)

            # 检查预测类别与实际类别的匹配情况
            correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            # 计算准确率
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
            #tf.summary.scalar('accuracy',accuracy)
            
            merged_summary_op = tf.summary.merge_all()
            writer = tf.summary.FileWriter('log/', sess.graph)
            #writer = tf.summary.FileWriter('C:/Users/cheng/Desktop/TFOD/Data_Science_Bowl_2017/log',sess.graph) #创建写入符
            

            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
            # 模型保存
            if (epoch+1)%save_step == 0:
                saver.save(sess , os.path.join(ckpt_dir,
                                            'model_Lung_cancer{:06d}.ckpt'.format(epoch+1)))
                print('model_Lung_cancer{:06d}.ckpt saved'.format(epoch+1))
                
                
                           
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        print('fitment percent:',successful_runs/total_runs)
                      

In [13]:
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 2 loss: 0
Accuracy: 1.0
model_Lung_cancer000001.ckpt saved
Epoch 2 completed out of 2 loss: 0
Accuracy: 1.0
model_Lung_cancer000002.ckpt saved
Done. Finishing accuracy:
Accuracy: 0.0
fitment percent: 0.0
